In [1]:
# %pip install torch tensorboardX tensorboard pandas seaborn

In [2]:
# %pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

In [3]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, SubsetRandomSampler
import torch.optim.lr_scheduler as lr_scheduler
from tensorboardX import SummaryWriter
import os
from time import perf_counter

In [4]:

class CustomDataSet(Dataset):
    def __init__(self, x : np.ndarray,y : np.ndarray):
        # x: numpy array
        # y: numpy array
        self.data = x
        self.label = y
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index : int):
        data = self.data[index]
        label = self.label[index]
        
        return data, label

In [5]:
class Net(nn.Module):
  def __init__(self,input_shape,layerWidth=[16,32],dropoutRate=0.3):
    """
    Extend of nn.Module
    Wide Neural Network 
    layerWidth: width of the hidden layers
    input_shape: shape of the input
    Regression
    """
    super(Net,self).__init__()
    self.dropoutRate = dropoutRate
    self.fc1 = nn.Linear(input_shape,layerWidth[0])
    self.fc2 = nn.Linear(layerWidth[0],layerWidth[1])
    self.fc3 = nn.Linear(layerWidth[1],1)
  def forward(self,x):
    # self.training is with nn.Module
    x = torch.relu(self.fc1(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.relu(self.fc2(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = self.fc3(x)
    return x

class NetWide(nn.Module):
  def __init__(self,input_shape,layerWidth=512,dropoutRate=0.3):
    """
    Extend of nn.Module
    Wide Neural Network 
    layerWidth: width of the hidden layers
    input_shape: shape of the input
    Regression
    """
    super(NetWide,self).__init__()
    self.dropoutRate = dropoutRate
    self.fc1 = nn.Linear(input_shape,layerWidth)
    self.fc2 = nn.Linear(layerWidth,1)
  def forward(self,x):
    # self.training is with nn.Module
    x = torch.relu(self.fc1(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = self.fc2(x)
    return x

class NetWideDeep(nn.Module):
  def __init__(self,input_shape,layerWidth=512,dropoutRate=0.3):
    """
    Extend of nn.Module
    Wide Deep Neural Network 
    layerWidth: width of the hidden layers
    input_shape: shape of the input
    Regression
    """
    super(NetWideDeep,self).__init__()
    self.dropoutRate = dropoutRate
    self.fc1 = nn.Linear(input_shape,layerWidth)
    self.fc2 = nn.Linear(layerWidth,layerWidth)
    self.fc3 = nn.Linear(layerWidth,layerWidth)
    self.fc4 = nn.Linear(layerWidth,1)
  def forward(self,x):
    # self.training is with nn.Module
    x = torch.relu(self.fc1(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.relu(self.fc2(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.relu(self.fc3(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.relu(self.fc4(x))
    return x

In [6]:
# path of CSV file
processedDataDirectoryPath = "processedData"
trainCSVPath = os.path.join(processedDataDirectoryPath,'processed_train_data.csv')
testCSVPath = os.path.join(processedDataDirectoryPath,'processed_test_data.csv')
# read CSV file
# split data and label
label = "Hardness"
df = pd.read_csv(trainCSVPath)
y = df[label]
df = df.drop(label, axis=1)
# convert to numpy array
x = df.values
y = y.values
print(x.shape)


(10407, 11)


In [7]:
# Hyperparameters
layerWidth2d = [256,256]
layerWidth = 64
lr = 0.00001
batchSize = 64
dropout = 0.0
epoch = 700
device = torch.device("mps")
modelName = "Normal"
modelList = {"Normal" : Net(x.shape[1], layerWidth=layerWidth2d,dropoutRate=dropout),
             "Wide" : NetWide(x.shape[1],layerWidth=layerWidth,dropoutRate=dropout),
             "WideDeep" : NetWideDeep(x.shape[1],layerWidth=layerWidth,dropoutRate=dropout)}
model = modelList[modelName]
# model = NetDeep(x.shape[1],dropoutRate=dropout)
model = model.to(device)
 
# May try otu different optimizers
# optimizer = optim.SGD(model.parameters(), lr=lr)
optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5)
# loss_fn = nn.BCELoss()
# loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()


validation_split = .2 # 20% of the data is used for validation
dataset = CustomDataSet(x,y)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,sampler=valid_sampler)
train_features, train_labels = next(iter(train_loader))
print(len(train_loader))
print(len(validation_loader))


131
33


In [8]:
def validationTest(model, val_loader):
    acc = 0
    loss = 0
    model.eval()
    with torch.no_grad():
        for (data, target) in val_loader:
            data = data.to(torch.float32)
            target = target.to(torch.float32)
            data = data.to(device)
            target = target.to(device)
            target = target.unsqueeze(1)
            output = model(data)
            loss = loss_fn(output,target)
            acc += (np.round(output.reshape(-1).detach().cpu().numpy()) == np.round(target.reshape(-1).detach().cpu().numpy())).mean()
        loss = loss.item()
    return acc/len(val_loader), loss
    

In [9]:
def train(model,epochs, model_path = "./model.pth",log_path = "log_pytorch"):
   
    tb = SummaryWriter(f"./train_{log_path}/")  # training tensorboard
    vtb = SummaryWriter(f"./val_{log_path}/")  # validation tensorboard

    # put the model into training mode
    model.train()
    outputFormat = "{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}"
    print(outputFormat.format("Mode","Epoch","Loss","Accuracy","Time Used","Progress","lr"))
    outputFormat = "{:^10}|{:^10.0f}|{:^10.4f}|{:^10.4f}|{:^10.2f}|{:^10}|{:^10.6f}"
    minValLoss = 100
    for epoch in range(1, epochs + 1):
        startTime = perf_counter()
        length = len(train_loader)
        acc = 0
        loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(torch.float32)
            target = target.to(torch.float32)
            data = data.to(device)
            target = target.to(device)
            target = target.unsqueeze(1)
            optimizer.zero_grad()
            #calculate output
            output = model(data)
            loss = loss_fn(output,target)
            acc += (np.round(output.reshape(-1).detach().cpu().numpy()) == np.round(target.reshape(-1).detach().cpu().numpy())).mean()
            #backprop
            loss.backward()
            optimizer.step()
            print(outputFormat.format("Train",epoch,loss.item(),acc/(batch_idx+1),perf_counter() - startTime,f"{batch_idx+1}/{length}",optimizer.param_groups[0]['lr']),end="\r")
        # print(outputFormat.format("Train",epoch,loss.item(),acc/(batch_idx+1),perf_counter() - startTime,f"{batch_idx+1}/{length}"))
        trainAcc = acc/len(train_loader)
        tb.add_scalar("epoch loss", loss.item(), epoch)
        tb.add_scalar("epoch accuracy", trainAcc, epoch)
        print(outputFormat.format("Train",epoch,loss.item(),trainAcc,perf_counter() - startTime,f"{batch_idx+1}/{length}",optimizer.param_groups[0]['lr']))

        val_Acc , val_Loss = validationTest(model,validation_loader)
        print(outputFormat.format("Validation",epoch,val_Loss,val_Acc,perf_counter() - startTime,"Done",optimizer.param_groups[0]['lr']))
        vtb.add_scalar("epoch loss", val_Loss, epoch)
        vtb.add_scalar("epoch accuracy", val_Acc, epoch)
        for name, weight in model.named_parameters():
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f'{name}.grad',weight.grad, epoch)
        scheduler.step()
        # save model if is the lowest validation loss
        if val_Loss < minValLoss:
            minValLoss = val_Loss
            print('Saving NN to %s' % model_path, "epoch: ", epoch)
            torch.save(model.state_dict(), model_path)
        # print(optimizer.param_groups[0]['lr'])
    # add graph to tensorboard
    tb.add_graph(model, (data,))

In [10]:
def modelPred(model,test_data,model_path="./model.pth"):

    model.load_state_dict(torch.load(model_path))
    
    # put model into test mode
    model.eval()
    with torch.no_grad():
    
        output = model(torch.tensor(test_data,dtype=torch.float32))
        # pred = np.round(output.reshape(-1).detach().numpy(),1)
        pred = output.reshape(-1).detach().numpy()
        
        return pred

In [11]:
strLR =  str(lr).replace(".","-")
strDropout = str(dropout).replace(".","-")
modelStr = f"{modelName}_0-1set_{batchSize}b_{strLR}lr_{layerWidth}w_{epoch}e_{strDropout}d"
modelPath = f"./{modelStr}.pth"
train(model,epochs=epoch,model_path=modelPath,log_path=modelStr)


   Mode   |  Epoch   |   Loss   | Accuracy |Time Used | Progress |    lr    
  Train   |    1     |  3.9072  |  0.0000  |   1.26   | 131/131  | 0.000010 
Validation|    1     |  4.4862  |  0.0000  |   1.35   |   Done   | 0.000010 
Saving NN to ./Normal_0-1set_64b_1e-05lr_64w_700e_0-0d.pth epoch:  1
  Train   |    2     |  3.7518  |  0.0001  |   0.92   | 131/131  | 0.000010 
Validation|    2     |  4.4430  |  0.0005  |   0.98   |   Done   | 0.000010 
Saving NN to ./Normal_0-1set_64b_1e-05lr_64w_700e_0-0d.pth epoch:  2
  Train   |    3     |  2.7132  |  0.0017  |   0.88   | 131/131  | 0.000010 
Validation|    3     |  3.6846  |  0.0014  |   0.94   |   Done   | 0.000010 
Saving NN to ./Normal_0-1set_64b_1e-05lr_64w_700e_0-0d.pth epoch:  3
  Train   |    4     |  4.5779  |  0.0024  |   0.90   | 131/131  | 0.000010 
Validation|    4     |  3.8365  |  0.0028  |   0.96   |   Done   | 0.000010 
  Train   |    5     |  3.3504  |  0.0699  |   0.89   | 131/131  | 0.000010 
Validation|    5     | 

In [12]:
# testModel = NetDeep(x.shape[1])
# testModel = NetWide(x.shape[1])
modelList = {"Normal" : Net(x.shape[1], layerWidth=layerWidth2d,dropoutRate=dropout),
             "Wide" : NetWide(x.shape[1],layerWidth=layerWidth,dropoutRate=dropout),
             "WideDeep" : NetWideDeep(x.shape[1],layerWidth=layerWidth,dropoutRate=dropout)}
testModel = modelList[modelName]
test_data = pd.read_csv(testCSVPath)
Id = "id"
y = test_data[Id]
test_data = test_data.drop(Id,axis=1)
predict = modelPred(testModel,test_data=test_data.values,model_path=modelPath)
print(predict)
mapped = {"id":y.values,"Hardness" : predict}

df = pd.DataFrame.from_dict(mapped)
print(df.shape)
df.to_csv("result.csv",index=False)

[2.7256072 4.6245255 5.9135904 ... 5.9380465 3.104553  1.5455712]
(6939, 2)
